In [1]:
from graphics import *
import random
import numpy as np
import time

#### Create the 5x5 Grid world environment 

In [2]:
win = GraphWin("My Window", 600,600)
win.setBackground(color_rgb(255,255,255))

In [3]:
def line(x1,y1,x2,y2):
    """
    create a line between pt(x1,y1) and pt(x2,y2) and return it
    """
    ln = Line(Point(x1,y1),Point(x2,y2))
    ln.setOutline(color_rgb(0,0,0))
    ln.setWidth(2)
    
    return ln

In [4]:
#border lines
border1 = line(50,50,550,50)
border1.draw(win)

border2 = line(550,50,550,550)
border2.draw(win)

border3 = line(50,550,550,550)
border3.draw(win)

border4 = line(50,50,50,550)
border4.draw(win)

Line(Point(50.0, 50.0), Point(50.0, 550.0))

In [5]:
#lines horizontal
ln1 = line(50,150,550,150)
ln1.draw(win)

ln2 = line(50,250,550,250)
ln2.draw(win)

ln3 = line(50,350,550,350)
ln3.draw(win)

ln4 = line(50,450,550,450)
ln4.draw(win)

Line(Point(50.0, 450.0), Point(550.0, 450.0))

In [6]:
#lines vertical

lnv1 = line(150,50,150,550)
lnv1.draw(win)

lnv2 = line(250,50,250,550)
lnv2.draw(win)

lnv3 = line(350,50,350,550)
lnv3.draw(win)

lnv4 = line(450,50,450,550)
lnv4.draw(win)

Line(Point(450.0, 50.0), Point(450.0, 550.0))

In [7]:
#create goal, obstacle and hole

rect_goal = Rectangle(Point(450,50), Point(550,150))
rect_goal.setFill(color_rgb(0,255,0))
rect_goal.draw(win)

rect_hole = Rectangle(Point(450,150), Point(550,250))
rect_hole.setFill(color_rgb(255,0,0))
rect_hole.draw(win)

rect_obs1 = Rectangle(Point(150,350), Point(250,450))
rect_obs1.setFill(color_rgb(0,0,0))
rect_obs1.draw(win)

rect_obs2 = Rectangle(Point(150,250), Point(250,350))
rect_obs2.setFill(color_rgb(0,0,0))
rect_obs2.draw(win)

Rectangle(Point(150.0, 250.0), Point(250.0, 350.0))

#### create agent looks like circle

In [8]:
agent = Circle(Point(100,500), 25)
agent.setFill(color_rgb(0,0,255))
agent.draw(win)

Circle(Point(100.0, 500.0), 25)

In [9]:
def up(agent):
    val = agent.getCenter()
    
    if (val.getY()) == 100 or (val.getY() == 500 and val.getX() == 200) :
        pass
    else :
        agent.move(0,-100)
        
    
def down(agent):
    val = agent.getCenter()
    
    if val.getY() == 500 or (val.getX() == 200 and val.getY() == 200):
        pass
    else :
        agent.move(0,100)

def right(agent):
    val = agent.getCenter()
    
    if val.getX() == 500 or (val.getX() == 100 and (val.getY() == 300 or val.getY() == 400)):
        pass
    else :
        agent.move(100,0)

def left(agent):
    val = agent.getCenter()
    
    if val.getX() == 100 or (val.getX() == 300 or (val.getY() == 300 or val.getY() == 400)):
        pass
    else :
        agent.move(-100,0)

In [10]:
def action(agent,val):
    if val == 0:
        up(agent)
    elif val == 1:
        down(agent)
    elif val == 2:
        right(agent)
    else :
        left(agent)

In [11]:
for i in range(200):
    val = random.choice([0,1,2,3])
    action(agent,val)

#### Reward for the agent in the each state

In [12]:
reward = np.array([[-1,-1,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-1,-1,-1,-100],
                   [-1,-1,-1,-1,100]])

In [13]:
def rewards(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    
    return reward[j][i]

In [14]:
def observation(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    return [i,j]

In [15]:
def env_close():
    win.close()

In [16]:
def reset(agent):
    agent.move(-400,400)

In [17]:
def set_agent(agent,obs):
    pos = agent.getCenter()
    
    pos = [pos.getX(), pos.getY()]
    
    refer_pos = [(obs[0]+1) * 100, (500 - (obs[1]*100))]
    
    
    agent.move(refer_pos[0]-pos[0], refer_pos[1] - pos[1])

#### Create the value function and policy

In [143]:
old_value = np.array([[.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,0.0],
                  [.0,.0,.0,.0,100.0]])

new_value = np.array([[.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,0.0],
                  [.0,.0,.0,.0,100.0]])

In [144]:
policy = np.zeros([5,5])

#assign random policy
for row in range(5):
    for col in range(5):
        policy[row,col] = random.choice([0,1,2,3])

#### Declare the variable as episode

In [171]:
policy_iteration = 100
policy_evaluation = 1
change_policy_after_every_episode = 50
discount_factor = 0.9
learning_rate = .01

In [172]:
for j in range(policy_iteration):
    
    #change policy after every 5 iteration and start value with zero
    if j != 100:
        #improve the policy depends upon maximum value of neighbour
        acti = [0,1,2,3]
        for row in range(5):
            for col in range(5):
                maxi = 0
                maxi_action = 0
                for act in acti:
                    set_agent(agent,[row,col])
                    action(agent,act)
                    
                    obser = observation(agent)
                    #select action gives maximum value neigbour 
                    if value[obser[0],obser[1]] > maxi:
                            maxi_action = act
                            maxi = new_value[obser[0],obser[1]]
                
                policy[row,col] = maxi_action
                    
        
    for i in range(policy_evaluation):
        
        #change value for all state
        for row in range(5):
            for col in range(5):
                
                    
                set_agent(agent,[row,col])
                Reward = rewards(agent)
                #choose the action
                act = policy[row,col]
                    
                action(agent,act)
                    
                new_state = observation(agent)
                
                i
                new_value[row,col] = Reward + (discount_factor * old_value[new_state[0],new_state[1]])
        
        new_value[4,4] = 100
        old_value = new_value
       
        

In [193]:
value[state[0],state[1]] = value[state[0],state[1]] + (learning_rate * ((rewards(agent) + (discount_factor*value[n_state[0],n_state[1]])-value[state[0],state[1]])))

In [174]:
value[state[0],state[1]] = value[state[0],state[1]] + (learning_rate * ((rewards(agent) + (discount_factor*value[state[0],state[1]])-value[state[0],state[1]])))

####  Initial random policy

|right|right| up  | up  | up  |
|-----|-----|-----|-----|-----|
|left | left|right| up  | up  |
|-----|-----|-----|-----|-----|
|left |     |right| up  | up  |
|-----|-----|-----|-----|-----|
| up  |     | up  |down |down |
|-----|-----|-----|-----|-----|
| up  |down |right|down | up  |

In [145]:
After 30 policy iteration

array([[0., 0., 3., 3., 2.],
       [1., 3., 1., 3., 2.],
       [2., 0., 2., 2., 0.],
       [1., 1., 0., 0., 0.],
       [0., 1., 2., 0., 0.]])

In [ ]:
| up  |right|right|right|     |
|-----|-----|-----|-----|-----|
| up  | up  | up  | up  | up  |
|-----|-----|-----|-----|-----|
| up  |     | up  | up  |left |
|-----|-----|-----|-----|-----|
|left |     | up  | up  | up  |
|-----|-----|-----|-----|-----|
| up  | up  | up  | up  | up  |

In [153]:
policy

array([[0., 3., 3., 2., 2.],
       [0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 2.],
       [0., 0., 3., 0., 3.]])

#### After 60 policy iteration

|right|right|right|right|     |
|-----|-----|-----|-----|-----|
|right| up  | up  | up  | up  |
|-----|-----|-----|-----|-----|
|left |     | up  | up  |left |
|-----|-----|-----|-----|-----|
|left |     | up  | up  | up  |
|-----|-----|-----|-----|-----|
| up  | up  | up  | up  | up  |

#### After 100 policy iteration

In [174]:
| up  |right|right|right|     |
|-----|-----|-----|-----|-----|
| up  | up  | up  | up  | up  |
|-----|-----|-----|-----|-----|
| up  |     | up  | up  |left |
|-----|-----|-----|-----|-----|
| up  |     | up  | up  | up  |
|-----|-----|-----|-----|-----|
| up  | up  | up  | up  | up  |

array([[ 32.61625379,  42.612659  ,  42.612659  ,  54.9539    ,
         54.9539    ],
       [-10.        ,  12.35851   ,  35.9539    ,  62.171     ,
         70.19      ],
       [ 48.45851   ,  54.9539    ,  62.171     ,  70.19      ,
         79.1       ],
       [ 54.9539    ,  62.171     ,  70.19      ,  79.1       ,
         89.        ],
       [-10.        , -10.        , -10.        , -10.        ,
        100.        ]])

In [178]:
new_value

array([[ 32.61625379,  42.612659  ,  42.612659  ,  54.9539    ,
         54.9539    ],
       [-10.        ,  12.35851   ,  35.9539    ,  62.171     ,
         70.19      ],
       [ 48.45851   ,  54.9539    ,  62.171     ,  70.19      ,
         79.1       ],
       [ 54.9539    ,  62.171     ,  70.19      ,  79.1       ,
         89.        ],
       [-10.        , -10.        , -10.        , -10.        ,
        100.        ]])